In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
ROOT = ".."
df_obs = pd.read_csv(f"{ROOT}/data_calculated/Carbon8_descriptor.csv", 
                     index_col=[0,1])
df_new = pd.read_csv(
        f"{ROOT}/data_calculated/Carbon8_descriptor_selected_sp.csv", 
        index_col=[0, 1])
DESCRIPTOR_NAMES = ['a0.25_rp1.0', 'a0.25_rp1.5', 'a0.25_rp2.0', 
                     'a0.25_rp2.5', 'a0.25_rp3.0', 'a0.5_rp1.0', 
                     'a0.5_rp1.5', 'a0.5_rp2.0', 'a0.5_rp2.5',
                     'a0.5_rp3.0', 'a1.0_rp1.0', 'a1.0_rp1.5', 
                     'a1.0_rp2.0', 'a1.0_rp2.5', 'a1.0_rp3.0']
SPLABEL="sp_label"
ans_list = df_new[SPLABEL].values

In [ ]:
from sklearn.preprocessing import StandardScaler
Xraw = df_obs[DESCRIPTOR_NAMES].values
scaler = StandardScaler()
scaler.fit(Xraw)
X = scaler.transform(Xraw)


In [ ]:
from sklearn.decomposition import PCA
drd = PCA(2)
drd.fit(X)
X_PCA = drd.transform(X)


In [ ]:
from sklearn.cluster import KMeans
NCLUSTERS = 3
km = KMeans(NCLUSTERS, random_state=1)
km.fit(X)
yp_km = km.predict(X)


In [ ]:
from clustering_misc import plot_X_withlabel
Xraw_new = df_new[DESCRIPTOR_NAMES].values
X_new = scaler.transform(Xraw_new)
X_new_PCA = drd.transform(X_new)
plot_X_withlabel(X_PCA, X_new_PCA, ans_list)


In [ ]:
from clustering_misc import plot_X_clusters
plot_X_clusters(X_PCA, yp_km, alpha=0.05)


In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(NCLUSTERS, random_state=3)
gmm.fit(X)
yp_gmm = gmm.predict(X)
plot_X_clusters(X_PCA, yp_gmm, alpha=0.01)

In [ ]:
from clustering_misc import make_df_sample
X_sample, ans_list_sample = make_df_sample(df_new, DESCRIPTOR_NAMES, 
                                           n=3, group_name=SPLABEL)


In [ ]:
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
metric = 'euclidean'
pairdistance = pdist(X_sample, metric=metric)
method = 'complete'
Z = linkage(pairdistance, method=method)


In [ ]:
from scipy.cluster.hierarchy import dendrogram
fig, ax = plt.subplots(figsize=(3, 5))
tree = dendrogram(Z, labels=ans_list_sample, orientation="left", ax=ax)
ax.invert_yaxis()
fig.tight_layout()
